### Load data

In [1]:
import numpy as np
import pandas as pd


def load_data(dataset, train = True):
    
    if train:
        df_Xtr = pd.read_csv(f'dataset/Xtr{dataset}.csv', index_col=0)   # read training X
        df_Ytr = pd.read_csv(f'dataset/Ytr{dataset}.csv')  # read training y
        
        X = np.array(df_Xtr).squeeze()
        Y = df_Ytr.Bound.values.ravel().astype(float)
        
    else:
        df_Xte = pd.read_csv(f'dataset/Xte{dataset}.csv', index_col=0)   # read test X
        X = np.array(df_Xte).squeeze()
        Y = None
    
    return X, Y 

### Construct dictionary and the gram matrix

In [2]:
def dic_constr(X, k):
    Subseqs_dic ={}
    
    for idx, fulseq in enumerate(X):
        # compute all its subsequences
        subseqs = [ fulseq[i:i + k]  for i in range(len(fulseq) - k + 1)  ]
        
        for subseq in subseqs:
            # creat a new dict for new subsequences
            if not subseq in Subseqs_dic:
                Subseqs_dic[subseq] = {}
            if not str(idx) in Subseqs_dic[subseq]:
                Subseqs_dic[subseq][str(idx)] = 0
            
            Subseqs_dic[subseq][str(idx)] += 1
                    
    return Subseqs_dic

In [3]:
def gram_matrix(dicS, dicT, nrow, ncol):
    # dicS : always the training dictionary
    # dicT : test dictionary (could be the training set)
    # nrow : size of test set (could be the training set)
    # ncol : always size of training set
    import time
    gram_mat = np.zeros((nrow,ncol))
    vetS = np.zeros(ncol)
    vetT = np.zeros(nrow)
    
    start = time.time()
    for subseqT, subdicT in dicT.items():
        if subseqT in dicS.keys():
            subdicS = dicS[subseqT]
            
            for i, numi in subdicT.items():
                for j, numj in subdicS.items():
                    gram_mat[int(i),int(j)] += numi * numj   
#                     vetS[int(j)] += numj**2
#                     vetT[int(i)] += numi**2

    for subseqT, subdicT in dicT.items():
        for i, numi in subdicT.items():
            vetT[int(i)] += numi**2
    for subseqS, subdicS in dicS.items():
        for j, numj in subdicS.items():
            vetS[int(j)] += numj**2
            
                    
    denom = np.sqrt(np.outer(vetT,vetS))
#    denom[denom == 0] = 1  # should not have 0 by the definition
    gram_mat = np.divide(gram_mat,denom)
    
    stop = time.time()
    print('\t time {:.4f}'.format(stop - start))
    
    return gram_mat

### SVM solution

In [4]:
def sign(y):
    return 2*y-1

def binary(y):
    return ((y + 1) / 2).astype(int)

In [5]:
# SVM solution
from cvxopt import solvers, matrix

def SVM(K, y, lbd=1):
#     print(lbd)
    y = sign(y)  # {0,1} to {-1,1}
    n = len(y)
    
    q = - 2 * y
    P = K
    G = np.zeros((2*n, n))
    G[:n, :] = - np.diag(y)
    G[n:, :] = np.diag(y)
    h = np.zeros(2 * n)
    h[n:] = 1 / (2 * lbd * n)
    
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)

    solvers.options['show_progress'] = False
    alpha = solvers.qp(P, q, G, h)

    return alpha

def predict(K, alpha):
    return binary(np.sign(K@alpha))

In [6]:
dataset = ['0','1','2']
for set in dataset:
    Xtr, Ytr = load_data(set,train =True)
    Xte,_ = load_data(set,train = False)

# length of subsequence
k = 20
dic_tr = dic_constr(Xtr, k)
dic_te = dic_constr(Xte, k)

len_tr, len_te = len(Xtr), len(Xte)

# the gram matrix 
print('Compute the training gram matrix...')
gram_mat_train = gram_matrix(dic_tr,dic_tr, len_tr,len_tr)
print('Compute the testing gram matrix...')
gram_mat_test = gram_matrix(dic_tr, dic_te, len_te,len_tr)

# SVM solution
sol = SVM(gram_mat_train, Ytr)
pred_train = predict(gram_mat_train, sol['x'])
acc_train = np.sum(np.abs(pred_train.squeeze() - Ytr)) / len_tr
acc_train = 1-acc_train
print('Length of subsequence is : {:}, and the accuracy on training set is {:.4f}'.format(k, acc_train))

# output on the test set
pred_test = predict(gram_mat_test, sol['x'] )

Compute the training gram matrix...
	 time 0.6826
Compute the testing gram matrix...
	 time 0.3185
Length of subsequence is : 20, and the accuracy on training set is 0.9970


In [10]:
len(Xtr[0])

101

In [7]:
def acc(Xtr, Ytr, Xte, Yte, k, lbd):
    
    dic_tr = dic_constr(Xtr, k)
    dic_te = dic_constr(Xte, k)

    len_tr, len_te = len(Xtr), len(Xte)

    # the gram matrix 
    print('Compute the training gram matrix...')
    gram_mat_train = gram_matrix(dic_tr,dic_tr, len_tr, len_tr)
    print('Compute the testing gram matrix...')
    gram_mat_test = gram_matrix(dic_tr, dic_te, len_te, len_tr)

    # SVM solution
    sol = SVM(gram_mat_train, Ytr, lbd=lbd)
    pred_train = predict(gram_mat_train, sol['x'])
    acc_train = np.sum(np.abs(pred_train.squeeze() - Ytr)) / len_tr
    acc_train = 1-acc_train
    print('Length of subsequence is : {:}'.format(k))
    print('and the accuracy on training set is {:.4f}'.format(acc_train))
    
    pred_test = predict(gram_mat_test, sol['x'] ).squeeze()
    acc_test = 1 - np.sum(np.abs(pred_test - Yte)) / len_te
    print('The accuracy on test set is {:.4f}'.format(acc_test))
    
    return acc_test

In [39]:
from sklearn.model_selection import KFold

dataset = ['2']
dic_list = []
for set in dataset:
    X, Y = load_data(set,train =True)
    # Xte, _ = load_data(set,train = False)
    
    print('dataset'+set)
    
    length = len(X)
    para_dict = {}
    k_list = [8, 9]
    lbd_list = [1e-5, 1e-4, 1e-3, 1e-2]
    #k_list = [8, 10]
    #lbd_list = [1]

    kf = KFold(n_splits = 5)

    for k in k_list:
        for lbd in lbd_list:
            accu = 0
            print(k ,lbd)
            if not (k,lbd) in para_dict: 
                para_dict[(k,lbd)] = 0
            for train_idx, test_idx in kf.split(X):
                Xtr, Ytr = X[train_idx], Y[train_idx]
                Xte, Yte = X[test_idx], Y[test_idx]

                accu += acc(Xtr, Ytr, Xte, Yte, k, lbd)
            accu_test = accu/5
            para_dict[(k,lbd)] = accu_test
            print('\t parameters len {:},lbd {:}, average accuracy {:.4f}'.format(k ,lbd, accu_test))
            
    dic_list.append(para_dict)

dataset2
8 1e-05
Compute the training gram matrix...
	 time 1.5376
Compute the testing gram matrix...
	 time 0.4731
Length of subsequence is : 8
and the accuracy on training set is 1.0000
The accuracy on test set is 0.7125
Compute the training gram matrix...
	 time 1.4268
Compute the testing gram matrix...
	 time 0.4730
Length of subsequence is : 8
and the accuracy on training set is 1.0000
The accuracy on test set is 0.7625
Compute the training gram matrix...
	 time 1.4805
Compute the testing gram matrix...
	 time 0.4497
Length of subsequence is : 8
and the accuracy on training set is 1.0000
The accuracy on test set is 0.7025
Compute the training gram matrix...
	 time 1.5013
Compute the testing gram matrix...
	 time 0.4752
Length of subsequence is : 8
and the accuracy on training set is 1.0000
The accuracy on test set is 0.7425
Compute the training gram matrix...
	 time 1.5591
Compute the testing gram matrix...
	 time 0.4885
Length of subsequence is : 8
and the accuracy on training se

Compute the training gram matrix...
	 time 0.9604
Compute the testing gram matrix...
	 time 0.3272
Length of subsequence is : 9
and the accuracy on training set is 0.9850
The accuracy on test set is 0.7000
Compute the training gram matrix...
	 time 1.0313
Compute the testing gram matrix...
	 time 0.3417
Length of subsequence is : 9
and the accuracy on training set is 0.9869
The accuracy on test set is 0.6925
	 parameters len 9,lbd 0.01, average accuracy 0.6915


In [35]:
para_dict1 = dic_list[0].copy()
para_dict1

{(6, 1e-05): 0.591,
 (6, 0.0001): 0.6275000000000001,
 (6, 0.001): 0.64,
 (8, 1e-05): 0.6355000000000001,
 (8, 0.0001): 0.643,
 (8, 0.001): 0.6389999999999999}

In [38]:
para_dict2 = dic_list[1].copy()
para_dict2

{(6, 1e-05): 0.61,
 (6, 0.0001): 0.633,
 (6, 0.001): 0.6405,
 (6, 0.01): 0.641,
 (8, 1e-05): 0.6140000000000001,
 (8, 0.0001): 0.6144999999999999,
 (8, 0.001): 0.6325,
 (8, 0.01): 0.6325}

In [41]:
para_dict3 = dic_list[2].copy()
para_dict3

{(8, 1e-05): 0.7224999999999999,
 (8, 0.0001): 0.7284999999999999,
 (8, 0.001): 0.6900000000000001,
 (8, 0.01): 0.7025,
 (9, 1e-05): 0.716,
 (9, 0.0001): 0.7224999999999999,
 (9, 0.001): 0.6944999999999999,
 (9, 0.01): 0.6914999999999999}

In [40]:
para_dict3 = para_dict.copy()
para_dict3

{(8, 0.01): 0.6875,
 (8, 0.1): 0.6975,
 (8, 1): 0.6925,
 (8, 10): 0.69,
 (12, 0.01): 0.65,
 (12, 0.1): 0.6475,
 (12, 1): 0.6475,
 (12, 10): 0.6475,
 (15, 0.01): 0.5525,
 (15, 0.1): 0.5525,
 (15, 1): 0.5525,
 (15, 10): 0.5525,
 (20, 0.01): 0.52,
 (20, 0.1): 0.52,
 (20, 1): 0.52,
 (20, 10): 0.52}

In [44]:
paras = {'0': [8 ,0.0001], '1':[6,0.01], '2':[8,0.0001]}

In [45]:
dataset = ['0','1','2']
pre_y = []
for set in dataset:
    Xtr, Ytr = load_data(set,train =True)
    Xte,_ = load_data(set,train = False)

    # length of subsequence
    k = paras[set][0]
    lbd = paras[set][1]
    
    dic_tr = dic_constr(Xtr, k)
    dic_te = dic_constr(Xte, k)

    len_tr, len_te = len(Xtr), len(Xte)

    # the gram matrix 
    print('Compute the training gram matrix...')
    gram_mat_train = gram_matrix(dic_tr,dic_tr, len_tr,len_tr)
    print('Compute the testing gram matrix...')
    gram_mat_test = gram_matrix(dic_tr, dic_te, len_te,len_tr)

    # SVM solution
    sol = SVM(gram_mat_train, Ytr,lbd)
    pred_train = predict(gram_mat_train, sol['x'])
    acc_train = np.sum(np.abs(pred_train.squeeze() - Ytr)) / len_tr
    acc_train = 1-acc_train
    print('Length of subsequence is : {:}, and the accuracy on training set is {:.4f}'.format(k, acc_train))

    # output on the test set
    pred_test = predict(gram_mat_test, sol['x'] ).squeeze()
    pre_y.append(pred_test)

Compute the training gram matrix...
	 time 2.0105
Compute the testing gram matrix...
	 time 1.0168
Length of subsequence is : 8, and the accuracy on training set is 0.9880
Compute the training gram matrix...
	 time 11.4195
Compute the testing gram matrix...
	 time 5.6302
Length of subsequence is : 6, and the accuracy on training set is 0.7925
Compute the training gram matrix...
	 time 2.1960
Compute the testing gram matrix...
	 time 1.1125
Length of subsequence is : 8, and the accuracy on training set is 0.9970


In [46]:
pre_y

[array([1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
        0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
        0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 

In [47]:
pred_all = np.array(pre_y).reshape(3000)
pred = pd.Series(pred_all.astype('int'), name="Bound")
pred.index = pd.Series(np.arange(len(pred_all)), name="Id")

In [48]:
pred.to_csv('SVM_spectrum_norm.csv')

In [54]:
with open('pred_normal_2.npy', 'wb') as f:
    np.save(f, pre_y[2])

In [31]:
k = 3
count_matrix = np.zeros((len(Xtr),4**k))
#gram_matrix = np.zeros((len(Xtr), len(Xtr)))
list_ = []
bases = ['A','G','C','T']
for i in range(4):
    for j in range(4):
        for k in range(4):
            string =''
            string = bases[i]+ bases[j] + bases[k]
            list_.append(string)
        
list_.sort()
for idx, fulseq in enumerate(Xtr):
    # subsequece 
    substrings = [fulseq[i:i + k]  for i in range(len(fulseq) - k + 1)  ]
    for subseq in substrings:
        n = list_.index(subseq)
        count_matrix[idx,n] += 1 

In [32]:
kernel = count_matrix @ count_matrix.T

In [33]:
sum(sum(abs(gram_mat_train - kernel)))

0.0

In [34]:
kernel

array([[ 800.,  532.,  597., ...,  692.,  707.,  780.],
       [ 532.,  866.,  674., ...,  727.,  557.,  571.],
       [ 597.,  674.,  856., ...,  581.,  669.,  489.],
       ...,
       [ 692.,  727.,  581., ...,  886.,  635.,  794.],
       [ 707.,  557.,  669., ...,  635.,  764.,  639.],
       [ 780.,  571.,  489., ...,  794.,  639., 1026.]])

In [35]:
gram_mat_train

array([[ 800.,  532.,  597., ...,  692.,  707.,  780.],
       [ 532.,  866.,  674., ...,  727.,  557.,  571.],
       [ 597.,  674.,  856., ...,  581.,  669.,  489.],
       ...,
       [ 692.,  727.,  581., ...,  886.,  635.,  794.],
       [ 707.,  557.,  669., ...,  635.,  764.,  639.],
       [ 780.,  571.,  489., ...,  794.,  639., 1026.]])

In [9]:
np.divide(0,0)

<ipython-input-9-20851cc7e1db>:1: RuntimeWarning: invalid value encountered in true_divide
  np.divide(0,0)


nan